In [13]:
!pip3 install canvasapi
!pip3 install pyinstaller

In [12]:
import canvasapi

In [11]:
API_TOKEN = "6936~sfusmUWOeQdmhxPUkpd0emeG0laShQCJZkRG2R86DzJUf2gfvVOQwrxCz9TZp83f"  # pls don't hack me 
API_URL = "https://k12.instructure.com/"

In [15]:
canvas = canvasapi.Canvas(API_URL, API_TOKEN)

In [16]:
assignments = canvas.get_assignments()

for assignment in assignments:
    print(assignment)

AttributeError: 'Canvas' object has no attribute 'get_assignments'